# Cylinder plots

In [1]:
# Imports
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
import matplotlib.patches as mpatches
from IPython.display import Image
from numpy.linalg import lstsq
import pandas as pd

Latex Macros
$\newcommand{\AA}{\unicode{xC5}}$

In [2]:
def combine_sims(data_list):
    # Convert to 2d if not already
    for ii,data in enumerate(data_list):
        if data.ndim == 1:
            data_list[ii] = data.reshape([len(data),1])
    
    # Determine required array size
    data_shape = array([shape(data) for data in data_list])
    longest = max(data_shape[0])
    
    # Create array
    arr = empty([longest,sum(data_shape[:,1])])
    # Initialize with nans
    arr[:] = nan
    
    # Fill with data
    current_column = 0
    for ii,data in enumerate(data_list):
        data_width = data_shape[ii,1]
        arr[:len(data),current_column:current_column+data_width] = data
        current_column += data_width
    
    # Return time and filled array
    return arr

In [3]:
# Base radii
base_radius = array([20,30,40,50,60,100])
colors = ['r','b','g','y','m','c','orange','grey','brown','k']
# Times at which droplets have ceased spreading
endtimes = [1.5,12,12,12,12,12]

In [4]:
old_dir = "$SCRATCH/droplet/results/Bob/Sub951By50/finalResults/avg/"
cyl100_dir = "$SCRATCH/droplet/results/Bob/Sub951By100/finalResults/New/avg/"
cyl60_dir = "$SCRATCH/droplet/results/Bob/Sub951By50/finalResults/avg/"
old_mrad = genfromtxt(old_dir+"monoEdge.txt")
old_brad = genfromtxt(old_dir+"bulkEdge.txt")
# Use Sub951By50_Cyl60A
cyl60_mrad = genfromtxt(cyl60_dir+"monoEdge.txt")
cyl60_brad = genfromtxt(cyl60_dir+"bulkEdge.txt")
# Use Sub951By100_Cyl100A
cyl100_mrad = genfromtxt(cyl100_dir+"monoEdge.txt")
cyl100_brad = genfromtxt(cyl100_dir+"bulkEdge.txt")

# Define time
t_all = old_mrad[:,0]/1e6
# Remove time from radius arrays
old_mrad = old_mrad[:,1:]
old_brad = old_brad[:,1:]

# Crop old values
#crop_old_mrad = logical_or.reduce([old_mrad<=0,old_mrad>350])
#crop_old_brad = logical_or.reduce([old_brad<=0,old_brad>350])
#old_mrad[crop_old_mrad] = nan
#old_brad[crop_old_brad] = nan

# Filter old values based on numerical derivative
filter_cutoff = 10
dec_diff_filter_mrad = vstack([zeros(shape(old_mrad[1])),diff(old_mrad,axis=0)]) < -filter_cutoff
inc_diff_filter_mrad = vstack([diff(old_mrad,axis=0),zeros(shape(old_mrad[1]))]) > filter_cutoff
up_lim_filter_mrad = old_mrad > 400
lo_lim_filter_mrad = old_mrad <= 0
whole_filter_mrad = logical_or.reduce([
        dec_diff_filter_mrad,
        inc_diff_filter_mrad,
        up_lim_filter_mrad,
        lo_lim_filter_mrad])

dec_diff_filter_brad = vstack([zeros(shape(old_brad[1])),diff(old_brad,axis=0)]) < -filter_cutoff
inc_diff_filter_brad = vstack([diff(old_brad,axis=0),zeros(shape(old_brad[1]))]) > filter_cutoff
up_lim_filter_brad = old_brad > 400
lo_lim_filter_brad = old_brad <= 0
whole_filter_brad = logical_or.reduce([
        dec_diff_filter_brad,
        inc_diff_filter_brad,
        up_lim_filter_brad,
        lo_lim_filter_brad])

# Apply filters
old_mrad[whole_filter_mrad] = nan
old_brad[whole_filter_brad] = nan

# Combine sims
mrad = combine_sims([old_mrad[:,1:-1],cyl60_mrad[:,2],cyl100_mrad[:,1]])
brad = combine_sims([old_brad[:,1:-1],cyl60_brad[:,2],cyl100_brad[:,1]])

/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: RuntimeWarning: invalid value encountered in less
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:28: RuntimeWarning: invalid value encountered in greater
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in greater
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in less_equal
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in less
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in greater
/home/oge1/.local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:39: RuntimeWarning: invalid value encountered in greater
/home/oge1/.local/miniconda3/lib/python3.5/site-pac

ValueError: could not broadcast input array from shape (1509,1) into shape (1000,1)

In [ ]:
# Linear least squares
def lin_ls(xx,yy):
    # Set up system
    A = array([[sum(xx**2),sum(xx)],[sum(xx),len(xx)]])
    b = array([sum(xx*yy),sum(yy)])
    
    # Solve system using numpy.linalg.lstsq
    mm,bb = lstsq(A,b)[0]
    
    return mm,bb

In [ ]:
cyl50 = old_mrad[:,5]
plot(t_all,cyl50)

In [ ]:
for i in range(0,shape(old_mrad)[1]):
    plot(t_all,old_mrad[:,i])
title('Old')
xlim(8,9)

In [ ]:
for i in range(1,shape(cyl60_mrad)[1]):
    plot(cyl60_mrad[:,0]/1e6,cyl60_mrad[:,i])
title('Sub951By50')

In [ ]:
for i in range(1,shape(cyl100_mrad)[1]):
    plot(cyl100_mrad[:,0]/1e6,cyl100_mrad[:,i])
title('Sub951By100')

In [ ]:
# Create figure
figure(figsize=[16,10])

base_radius = [20,30,40,50,60,100]

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = t_all >= 0
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    plot(tt,mr,label=lbl)
title('Cylindrical Monolayer Radius')
xlabel('Time (ns)')
ylabel(r'Monolayer Radius ($\AA$)')
#ylim(0,400) 
#xlim(0,1)
#ylim(0,110)
legend(loc='best')

In [ ]:
# Create figure
figure(figsize=[16,10])

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = logical_and.reduce([isfinite(mrad[:,0]),t_all > 0])
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    plot(tt,mr**2,label=lbl)
title('Squared Cylindrical Monolayer Radius')
xlabel('Time (ns)')
ylabel(r'Monolayer Radius Squared ($\AA$)')
#ylim(0,400) 
xlim(0,10)
legend(loc='best')

In [ ]:
# Create figure
figure(figsize=[16,10])

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = t_all > 0
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    plot(log(tt),log(mr),
        label=lbl,color=colors[ii])
title('Double Log Cylindrical Monolayer Radius')
xlabel('Log Time (ns)')
ylabel(r'Log Monolayer Radius ($\AA$)')
#ylim(0,400) 
xlim(-2,log(12))
legend(loc='best')

In [ ]:
# Create figure
figure(figsize=[16,10])

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = logical_and.reduce([t_all<endtimes[ii],t_all > .1,isfinite(mrad[:,ii])])
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    
    # least sqares
    mm,bb = lin_ls(log(tt),log(mr))
    
    # Plot data
    plot(log(tt),log(mr),
        label=lbl,color=colors[ii])
    
    # Plot fit
    plot(log(tt),mm*log(tt)+bb,'--',label='fit_{}'.format(lbl),color=colors[ii])
title('Double Log Cylindrical Monolayer Radius')
xlabel('Log Time (log ns)')
ylabel(r'Log Monolayer Radius (log $\AA$)')
#ylim(0,400) 
#xlim(0,10)
legend(loc='best')

In [ ]:
# Create figure
figure(figsize=[16,10])

# Matrix of least squares coefficients
ls_coefs = zeros([6,2])

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = logical_and.reduce([t_all<endtimes[ii],t_all > .5,isfinite(mrad[:,ii])])
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    
    # least sqares
    mm,bb = lin_ls(log(tt),log(mr))
    
    # Save coefficients
    ls_coefs[ii,:] = mm,bb
    
    # Plot data
    plot(tt,mr,'o-',
        label=lbl,color=colors[ii])
    
    # Plot fit
    plot(tt,exp(bb)*tt**mm,'--',label='fit_{}'.format(lbl),color=colors[ii])
title('Cylindrical Monolayer Radius')
xlabel('Time (ns)')
ylabel(r'Monolayer Radius ($\AA$)')
#ylim(0,400) 
#xlim(0,10)
legend(loc='best')

In [ ]:
# Scaling coefficients (Ignore 20A)
start = 1
sc_m_m,sc_m_b = lin_ls(log(base_radius[start:]),log(ls_coefs[start:,0]))
sc_b_m,sc_b_b = lin_ls(log(base_radius[start:]),log(ls_coefs[start:,1]))

In [ ]:
plot(base_radius,ls_coefs[:,1],'o-',label='b')
plot(base_radius,ls_coefs[:,0],'o-',label='m')
title('Plot of coefficients')
legend()

In [ ]:
figure(figsize=[10,5])

subplot(1,2,1)
plot(log(base_radius),log(ls_coefs[:,1]),'o-',label='b')
plot(log(base_radius),sc_b_m*log(base_radius)+sc_b_b,'or-',label='fit')
title('Double log plot of coefficients: b')
legend()

subplot(1,2,2)
plot(base_radius,ls_coefs[:,1],'o-',label='b')
plot(base_radius,exp(sc_b_b)*base_radius**sc_b_m,'or-',label='fit')
title('Plot of coefficients: b')
legend()

In [ ]:
figure(figsize=[10,5])

subplot(1,2,1)
plot(log(base_radius),log(ls_coefs[:,0]),'o-',label='m')
plot(log(base_radius),sc_m_m*log(base_radius)+sc_m_b,'or-',label='fit')
title('Double log plot of coefficients: m')
legend()

subplot(1,2,2)
plot(base_radius,ls_coefs[:,0],'o-',label='m')
plot(base_radius,exp(sc_m_b)*base_radius**sc_m_m,'or-',label='fit')
title('Plot of coefficients: m')
legend()

In [ ]:
# Test model

# Create figure
figure(figsize=[16,10])

# Plot monolayer radius
for ii in range(0,mrad.shape[1]):
    lbl = '{}A'.format(base_radius[ii])
    whichvals = logical_and(t_all > 5e-1,isfinite(mrad[:,ii]))
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    
    # Scaling coefficients (non-log)
    #mm = sc_m_m*R[i-1] + sc_m_b
    #bb = sc_b_m*R[i-1] + sc_b_b
    
    # Scaling coefficients (log)
    mm = exp(sc_m_b)*base_radius[ii]**sc_m_m
    bb = exp(sc_b_b)*base_radius[ii]**sc_b_m
    
    # Model
    yy = exp(bb)*tt**mm
    
    # Plot data
    plot(tt,mr,
        label=lbl,color=colors[ii])
    
    # Plot model
    plot(tt,yy,'--',label='fit_{}'.format(lbl),color=colors[ii])
    
# Include 70A - 100A model
for ii in range(7,10):
    lbl = '{}A'.format(10*ii)
    tt = t_all[t_all > 5e-1]
    
    # Scaling coefficients (non-log)
    #mm = sc_m_m*R[i-1] + sc_m_b
    #bb = sc_b_m*R[i-1] + sc_b_b
    
    # Base radius
    RR = 10*(ii)
    # Scaling coefficients (log)
    mm = exp(sc_m_b)*RR**sc_m_m
    bb = exp(sc_b_b)*RR**sc_b_m
    
    # Model
    yy = exp(bb)*tt**mm
    
    # Plot model
    plot(tt,yy,'--',label='fit_{}A'.format(RR),color=colors[ii])

title('MODEL Spherical Monolayer Radius')
xlabel('Time (ns)')
ylabel(r'Monolayer Radius ($\AA$)')
#ylim(0,400) 
#xlim(0,10)
legend(loc='best')

In [ ]:
plot(log(R),log(ls_coefs[:,0]),'o-',label='m')
plot(log(R),log(ls_coefs[:,1]),'o-',label='b')
title('Double Log plot of coefficients')

In [ ]:
## Monolayer - Bulk Radius difference ##

# Create figure
figure(figsize=[16,10])

# Plot old Sub951By50 values
for ii in range(0,brad.shape[1]):
    whichvals = logical_and(t_all > 1e-1,isfinite(mrad[:,ii]))
    lbl = '{}A'.format(base_radius[ii])
    tt = t_all[whichvals]
    mr = mrad[:,ii][whichvals]
    plot(tt,mrad[:,ii][whichvals]-brad[:,ii][whichvals],'o-',
        label=lbl)

# Disclaimer
disclaimer = '''\
***************************
* Some extraneous values  *
* have been intentionally *
* removed from Old sims   *
***************************'''
#text(.2,60,disclaimer,ha='left',va='top',
    #family='monospace')

# Plot features
title('Summary of Cyl. Mono - Bulk Radii (as of 12/31/2016)')
xlabel('Time (ns)')
ylabel(r'Difference in radius ($\AA$)')
plot([0,1],[5,5],'k-')
#ylim(0,20) 
#xlim(0,.5)
legend(loc='best')
savefig('results/sph_diff.png')

In [ ]:
# Plot bulk and monolayer radius together for Sub951By50 Cyl50A

figure(figsize=[16,10])

# Cyl50A
i = 5

# bulk
#b_whichvals = logical_and(old_brad[:,i]<300,old_brad[:,i]>10)
b_lbl = 'Sub951By50 Cyl{}0A Bulk'.format(i)
plot(old_brad[:,0][whichvals]/1e6,old_brad[:,i][whichvals],
    label=b_lbl)

# mono
#m_whichvals = logical_and(old_mrad[:,i]<300,old_mrad[:,i]>10)
m_lml = 'Sub951By50 Cyl{}0A Mono'.format(i)
plot(old_mrad[:,0][whichvals]/1e6,old_mrad[:,i][whichvals],
    label=m_lml)

legend()
ylim(0,250)
xlim(0,10)